# Simulation

The goal of this notebook is to simulate the index mis-assignment encountered in sequencing, and validate the formulae used in the main program against the simulation truth.

### Index example
These indexes have been used for a project at the NSC facility (just an example).

In [1]:
index1s = split("""CTCGACTT CGAAGTAT TAGCAGCT TCTCTATG GATCTACG GTAACGAG ACGTGCGC ATAGTACC GCGTATAC AACGCTGA CGTAGCGA CTCGACTT TAGCAGCT TCTCTATG GATCTACG GTAACGAG ACGTGCGC ATAGTACC GCGTATAC TGCTCGTA AACGCTGA CGTAGCGA CTCGACTT CGAAGTAT TAGCAGCT TCTCTATG ACGTGCGC ATAGTACC GCGTATAC TGCTCGTA CGTAGCGA CTCGACTT TAGCAGCT TCTCTATG GATCTACG GTAACGAG ACGTGCGC ATAGTACC GCGTATAC TGCTCGTA AACGCTGA CGTAGCGA CTCGACTT TAGCAGCT GATCTACG GTAACGAG ATAGTACC GCGTATAC TGCTCGTA AACGCTGA CGTAGCGA CTCGACTT CGAAGTAT TCTCTATG GATCTACG GTAACGAG ACGTGCGC ATAGTACC GCGTATAC TGCTCGTA CGTAGCGA CTCGACTT CGAAGTAT TAGCAGCT TCTCTATG GATCTACG GTAACGAG ATAGTACC GCGTATAC TGCTCGTA AACGCTGA CGTAGCGA CTCGACTT CGAAGTAT TAGCAGCT TCTCTATG GATCTACG GTAACGAG CGTAGCGA CGAGAGTT GACATAGT ACGCTACT ACTCACTG TGAGTACG CTGCGTAG TAGTCTCC CGAGCGAC ACTACGAC GTCTGCTA GTCTATGA TATAGCGA CGAGAGTT GACATAGT ACGCTACT ACTCACTG TGAGTACG CTGCGTAG TAGTCTCC CGAGCGAC ACTACGAC GTCTGCTA GTCTATGA TATAGCGA CGAGAGTT GACATAGT ACGCTACT ACTCACTG TGAGTACG CTGCGTAG TAGTCTCC CGAGCGAC ACTACGAC GTCTGCTA GTCTATGA TATAGCGA CGAGAGTT GACATAGT ACGCTACT ACTCACTG TGAGTACG CTGCGTAG TAGTCTCC CGAGCGAC ACTACGAC GTCTGCTA GTCTATGA TATAGCGA CGAGAGTT GACATAGT ACGCTACT ACTCACTG TGAGTACG CTGCGTAG TAGTCTCC CGAGCGAC ACTACGAC GTCTGCTA GTCTATGA TATAGCGA CGAGAGTT GACATAGT ACGCTACT ACTCACTG TGAGTACG CTGCGTAG TAGTCTCC CGAGCGAC ACTACGAC GTCTGCTA GTCTATGA TATAGCGA CGAGAGTT GACATAGT ACTCACTG TGAGTACG CTGCGTAG TAGTCTCC CGAGCGAC ACTACGAC TATAGCGA TATAGCGA""", " ")
index2s = split("""CTACTATA CTACTATA CTACTATA CTACTATA CTACTATA CTACTATA CTACTATA CTACTATA CTACTATA CTACTATA CTACTATA CGTTACTA CGTTACTA CGTTACTA CGTTACTA CGTTACTA CGTTACTA CGTTACTA CGTTACTA CGTTACTA CGTTACTA CGTTACTA AGAGTCAC AGAGTCAC AGAGTCAC AGAGTCAC AGAGTCAC AGAGTCAC AGAGTCAC AGAGTCAC AGAGTCAC TACGAGAC TACGAGAC TACGAGAC TACGAGAC TACGAGAC TACGAGAC TACGAGAC TACGAGAC TACGAGAC TACGAGAC TACGAGAC ACGTCTCG ACGTCTCG ACGTCTCG ACGTCTCG ACGTCTCG ACGTCTCG ACGTCTCG ACGTCTCG ACGTCTCG TCGACGAG TCGACGAG TCGACGAG TCGACGAG TCGACGAG TCGACGAG TCGACGAG TCGACGAG TCGACGAG TCGACGAG GATCGTGT GATCGTGT GATCGTGT GATCGTGT GATCGTGT GATCGTGT GATCGTGT GATCGTGT GATCGTGT GATCGTGT GATCGTGT GTCAGATA GTCAGATA GTCAGATA GTCAGATA GTCAGATA GTCAGATA GTCAGATA CTACTATA CTACTATA CTACTATA CTACTATA CTACTATA CTACTATA CTACTATA CTACTATA CTACTATA CTACTATA CTACTATA CTACTATA CGTTACTA CGTTACTA CGTTACTA CGTTACTA CGTTACTA CGTTACTA CGTTACTA CGTTACTA CGTTACTA CGTTACTA CGTTACTA CGTTACTA AGAGTCAC AGAGTCAC AGAGTCAC AGAGTCAC AGAGTCAC AGAGTCAC AGAGTCAC AGAGTCAC AGAGTCAC AGAGTCAC AGAGTCAC AGAGTCAC TACGAGAC TACGAGAC TACGAGAC TACGAGAC TACGAGAC TACGAGAC TACGAGAC TACGAGAC TACGAGAC TACGAGAC TACGAGAC TACGAGAC ACGTCTCG ACGTCTCG ACGTCTCG ACGTCTCG ACGTCTCG ACGTCTCG ACGTCTCG ACGTCTCG ACGTCTCG ACGTCTCG ACGTCTCG ACGTCTCG TCGACGAG TCGACGAG TCGACGAG TCGACGAG TCGACGAG TCGACGAG TCGACGAG TCGACGAG TCGACGAG TCGACGAG TCGACGAG TCGACGAG GATCGTGT GATCGTGT GATCGTGT GATCGTGT GATCGTGT GATCGTGT GATCGTGT GATCGTGT GATCGTGT GTCAGATA""", " ")


indexes = zip(index1s, index2s) ;

### True number of reads
This code generates a set of true

In [2]:
sample_read_mean = 1e5               # Number of reads
sample_read_sd = sample_read_mean*0.5 # Statistical standard deviation (normal dist.)

true_sample_reads = Dict([index => (randn()*sample_read_sd + sample_read_mean) for index = indexes]) ;

### Simulate number of reads
The code below generates read multiplicities for each index combination, in accordance with what would be expected for a set of mis-assignment probabilities. The simulation logic assumes that the index1 and index2 mis-assignment events are independent.  Wright and Vetsigian found a higher probability than what would be expected for independent variables, but the double mis-assignment rate is still very low -- negligible for practical purposes:

|                  |              |
|------------------|--------------|
|Incorrect i5:     |   0.0604 %   |
|Incorrect i7:     |   0.0955 %   |
|Incorrect sequence|   0.0872 %   |
|Multiple incorrect|   0.0003 %   |

It is also notable that the i7 and i5 (respectively, the first and second index) mis-assignment rates are significantly different.

We cannot detect sequence misassignments at all using demultiplexing data, and most double misassignments will not be detectable.

### Simulation parameters (part 2)

In [3]:
index1_misassign_rate = 0.0955 / 100.0
index2_misassign_rate = 0.0604 / 100.0
## sequence_misassign_rate = 0.0872 / 100.0 ; #Not simulated

0.000604

### Possible indexes in the soup
The following arrays the read multiplicity of each single index in the data. I.e. if a sample has a large fraction of the reads, one expect the "free" indexes available for mis-assignment to also be relatively larger. Whether this is actually a valid model depends on the mechanism of mis-assigment.

In [4]:
index1_mult = Dict()
index2_mult = Dict()

for ((index1, index2), num_reads) in true_sample_reads
    index1_mult[index1] = get(index1_mult, index1, 0) + num_reads
    index2_mult[index2] = get(index2_mult, index2, 0) + num_reads
end

### Simulated number of reads per index sequence
This is the main simulation loop, which generates read counts per sample. It's a bit inefficient, could be optimised to not loop over each read.

In [5]:
println("Index 1 miss rate: ", index1_misassign_rate)
println("Index 2 miss rate: ", index2_misassign_rate)

Index 1 miss rate: 0.000955
Index 2 miss rate: 0.000604


In [6]:
read_norm = sum([reads for (seq, reads) = true_sample_reads])

1.6309883890471304e7

In [7]:
sim_index_reads = Dict()

function weightedrandom(norm, read_dict)
    value = rand() * norm
    for (seq, num) in read_dict
        value -= num
        if (value <= 0) 
            return seq
        end
    end
    throw(ErrorException("What happened?"))
end

for ((index1, index2), num_reads) in true_sample_reads
    for i = 1:num_reads
        if rand() < index1_misassign_rate
            index1var = weightedrandom(read_norm, index1_mult)
        else
            index1var = index1
        end
        if rand() < index2_misassign_rate
            index2var = weightedrandom(read_norm, index2_mult)
        else
            index2var = index2
        end
        index = (index1var, index2var)
        index_reads = get(sim_index_reads, index, 0)
        sim_index_reads[index] = index_reads + 1
    end
end
#sim_index_reads

#### Test case for weightedrandom
Does it reproduce the distribution?

In [8]:
println("Original index1 multiplicity (fraction): ")
for (seq, num) = index1_mult
    println(seq, ": ", num / read_norm)
end
# Generate weighted random, see if we get the same
wr_num = Dict()
N=1e5
for i = 0:N
    seq = weightedrandom(read_norm, index1_mult)
    wr_num[seq] = get(wr_num, seq, 0) + 1
end

println("Randomised index1 multiplicity (fraction): ")
for (seq, num) = wr_num
    println(seq, ": ", num / N)
end

Original index1 multiplicity (fraction): 
GCGTATAC: 0.04739235787817703
ACTACGAC: 0.04169314982576695
GTCTATGA: 0.041877731400489115
GACATAGT: 0.04487992840562026
AACGCTGA: 0.027653469687107763
GATCTACG: 0.04494778248733588
CGAGCGAC: 0.059296880332020616
GTAACGAG: 0.052305833708371934
TATAGCGA: 0.04537389794835174
GTCTGCTA: 0.03948643175645307
ATAGTACC: 0.05341831129818682
TGAGTACG: 0.041265861160900544
CGAAGTAT: 0.020408966464913823
CGAGAGTT: 0.05429352137687657
TCTCTATG: 0.029930055720464074
CGTAGCGA: 0.046545293784123344
ACGTGCGC: 0.025952009568437857
TGCTCGTA: 0.03449119232157036
TAGTCTCC: 0.025348275750347992
ACGCTACT: 0.03278092645256085
CTCGACTT: 0.0415520035300043
CTGCGTAG: 0.05250041275395844
TAGCAGCT: 0.04560833306528965
ACTCACTG: 0.050997373322671034
Randomised index1 multiplicity (fraction): 
GCGTATAC: 0.04752
ACTACGAC: 0.04181
GTCTATGA: 0.04225
GACATAGT: 0.04546
CGAGCGAC: 0.05916
GATCTACG: 0.04601
GTAACGAG: 0.05143
AACGCTGA: 0.02695
TATAGCGA: 0.04464
TGAGTACG: 0.04087
ATAG

# Setup for analysis
## Indexes

In [9]:
permutations = Set([(i1, i2) for i1 = index1s, i2 = index2s])
known = Set(indexes) ;
unknown = setdiff(permutations, known)
length(unknown)

31

## Metrics
Generate metrics that would be available in a real run.

### 1. Naive formula (wrong)
This counts the number of reads assigned to unknown index combinations compared the total number of reads. This ratio is in general less than the total mis-assignment rate, because there is also cross contamination between the actual sample indexes.

In [10]:
sum_sample_reads = sum([sim_index_reads[k] for k = known])
sum_unknown_reads = sum([sim_index_reads[k] for k = unknown])

2706

In [11]:
mis_id_rate_avg = sum_unknown_reads / (sum_unknown_reads + sum_sample_reads)
@printf("Average (simple) miss rate: %5.3f %%\n", mis_id_rate_avg*100)
trueone = (index1_misassign_rate + index2_misassign_rate) / 2.0
@printf("for comparison: true avg. miss rate: %5.3f %%\n", trueone*100)

Average (simple) miss rate: 0.017 %
for comparison: true avg. miss rate: 0.078 %


### Per index (end) analysis
This repeats the above analysis to see if there is a difference between the first and the second index.

In [12]:
i1known = 0
i1unknown = 0
for i1 in Set([i1 for (i1, i2)=keys(sim_index_reads)])
    i1known   += sum([sim_index_reads[k] for k=filter(x -> x[1] == i1, known)])
    i1unknown += reduce(+, 0, [sim_index_reads[k] for k=filter(x -> x[1] == i1, unknown)])
end
@printf("Index1 ratio: %5.3f %%\n", i1unknown * 100.0 / (i1known + i1unknown))
i2known = 0
i2unknown = 0
for i2 in Set([i2 for (i1, i2)=keys(sim_index_reads)])
    i2known   += sum([sim_index_reads[k] for k=filter(x -> x[2] == i2, known)])
    i2unknown += reduce(+, 0, [sim_index_reads[k] for k=filter(x -> x[2] == i2, unknown)])
end
@printf("Index2 ratio: %5.3f %%\n", i2unknown * 100.0 / (i2known + i2unknown))

Index1 ratio: 0.017 %
Index2 ratio: 0.017 %


## 2. Analysis
This is a more complex analysis taking into account the level of cross contaminations between samples, which are not observable. This is absolutely necessary to get a reliable result, otherwise only a fraction of the mis-assignment rate is included.

## Second derivation attempt
Let $a$ and $b$ represent particular first and second indexes respectively, and $N$ in general represent the number of fragments with a specific combination of indexes.

The following is an expression for the number of reads for a given index: 

$$ R_{a,b} \approx N^{true}_{a,b}-\alpha ( \sum_{i\neq a} N^{mis}_{a,b\to i,b}+\sum_{i\neq b} N^{mis}_{a,b\to a,i} )
   + \sum_{j\neq a} N^{mis}_{j,b\to a,b} + \sum_{i\neq b} N^{mis}_{a,i\to a,b}  \quad (1)$$

(ignoring read errors affecting single bases, etc.) The sums are over all possible indexes at the relevant position (first or second), regardless of whether the resulting combinations correspond to a real sample.

None of the terms on the right hand side are available as measurements. The first term represents the true number of fragments. The second and third terms (which are multiplied by $\alpha$) subtract number of times the true index $a,b$ was read as something else, either (second term) due to a wrong first index or (third term) a wrong second index. $\alpha$ itself is between zero and one and depends on the mode of mis-identification. If additional erroneous fragments are *created*, without destroying any of the original fragments, $\alpha$ is 0. This is true in the mode postulated by Sinha et al. If fragments instead are changed from the original index to a new one, $\alpha$ is 1.

The final two terms add back the number of times another index was mis-read as $a,b$. The above formula is an approximation because it ignores when *both* indexes were mis-read.

### Precise description of the terms
 * $ N^{true}_{a,b} $: Number of true fragments with indexes $a$ and $b$ that are detected, but are not necessarily read as such. $N^{true}_{a,b}$ includes an overall efficiency factor, which is the probability that a fragment will be detected at all. It only counts detected fragments.
 * $ N^{mis}_{a,b\to c,d}$: Number of fragments with index read as $c,d$, but which actually had a true index $a,b$ (where $a$ and $c$, $b$ and $d$, may be equal or different, but at least one of the two pairs is different).

These definitions are not directly useful in practice. We can measure $R_{a,b}$ for all combinations of indexes -- known and unknown (to a good approximation). We are, however, after the mis-identification probability, $\gamma$.

### Relating to the mis-identification probability "$\gamma$"
We have to construct a model of how the mis-identification probability relates to $N^{mis}_{a,b\to c,d}$. We would like to define a $\gamma$ in a way that is independent of the specific experiment setup used, i.e. the choice of indexes for each sample, or the relative number of reads for each sample. It should only depend on the particular sample prep and/or sequencing technology used (and potentially global conditions such as the error rate). There is in general a separate mis-ID probability for the first and the second index, $\gamma_1$ and $\gamma_2$.

We first concentrate on the specific case of a mis-identification of the second index, however this could be reversed in general. We can postulate that the number of mis-identified fragments originating from a particular index pair $a,j$ is proportional to the number of true such fragments:
$$ N^{mis}_{a,j\to a,k} = \gamma_2 N^{true}_{a,j} f(k)\; ,$$
where $f(k)$ is an unspecified function of the "replacement" index $k$. For mis-identification of the first index:
$$ N^{mis}_{j,b\to k,b} = \gamma_1 N^{true}_{j,b} f'(k)$$
($f'(k)$ is the corresponding function for the first index)

We know that index read mis-assignments are not caused by random read errors of single bases. Instead, there will exist a genuine index $k$ somewhere which somehow gets combined with $a$.

#### Functional dependence on the "target" index
In a model where true fragments with index $x,k$ (any first index) are consumed in order to convert $a,j$ to $a,k$, the function $f(k)$ would depend on the number of available such fragments, i.e.,
$$ f(k) = C \sum_i N_{i,k}^{true} \; .$$
One may absorb the constant of proportionality $C$ into $\gamma$, but we then label it $\gamma'$ to signify that it is no longer a probability.

The studies of Sinha et al. give us reason to believe that the probability to create new fragments with index $k$ depends on the amount of free index primer with the index $k$ left in the pool at the time of cluster generation. It is not given that the factorisation of $N^{mis}_{a,j\to a,k}$ above holds when the index primer is the limiting element.

### Solving for $\gamma$
We will show later how $\gamma$ can be a useful quantity to describe the scale of the mis-assignment problem, but we will first work out its relation to the observable quantities $R_{a,b}$.

In order to estimate $\gamma$, there needs to be an index combination $a,b$ which does not correspond to any sample -- a first and second index which exists in the pool, but not as a combination. Then we have $N^{true}_{a,b}=0$. Then Eq. 1 reduces to:
$$ R_{a,b} = -\alpha \left( \sum_{i\neq a} N^{mis}_{a,b\to i,b}+\sum_{i\neq b} N^{mis}_{a,b\to a,i} \right)
    + \sum_{i\neq a} N^{mis}_{i,b\to a,b} + \sum_{i\neq b} N^{mis}_{a,i\to a,b}  $$

Expanding the mis-identification terms leads to: 
$$ R_{a,b} = -\alpha \left( \sum_{i\neq a} (\gamma_1'  N_{a,b}^{true}\sum_m N_{m,i}^{true})
        + \sum_{i\neq b}  (\gamma_2' N_{a,b}^{true}\sum_m N_{i,m}^{true}) \right) \\
        + \sum_{i\neq a}  (\gamma_1' N_{a,i}^{true}\sum_m N_{m,b}^{true})
        + \sum_{i\neq b}  (\gamma_2' N_{i,b}^{true}\sum_m N_{a,m}^{true}) $$

The first two terms also vanish, because $N^{true}_{a,b}=0$.

$$ R_{a,b} = \sum_{i\neq a}  (\gamma_1' N_{a,i}^{true}\sum_m N_{m,b}^{true})
           + \sum_{i\neq b}  (\gamma_2' N_{i,b}^{true}\sum_m N_{a,m}^{true}) \\
           $$

The sums can be collected 
$$ R_{a,b} = \gamma_1' \sum_{i\neq a} \sum_m N_{a,i}^{true} N_{m,b}^{true}
           + \gamma_2' \sum_{i\neq b} \sum_m N_{i,b}^{true} N_{a,m}^{true} \; .
           $$
This is as far as we get with algebra.

### An approximate expression related to observable quantities
To first order in $\gamma$, $R_{i,j} = N_{i,j}^{true}$. Assuming the mis-identification probability is small, we can replace $N^{true}$ in the above expression:

$$ 
   R_{a,b} = \gamma_1' \sum_{i\neq a} \sum_m R_{a,i} R_{m,b}
           + \gamma_2' \sum_{i\neq b} \sum_m R_{i,b} R_{a,m} \; . \quad (2)
$$

$a,b$ is still assumed to be a specific index combination with zero true reads. For a moment we can assume that the mis-ID probabilities are equal (as well as the constants $C$), $\gamma_1'=\gamma_2'\equiv \gamma'$, and solve Eq. 2 for $\gamma'$:

$$
    \frac{R_{a,b}}{\sum_{i\neq a} \sum_m R_{a,i} R_{m,b} + \sum_{i\neq b} \sum_m R_{i,b} R_{a,m}} = \gamma' 
$$

In [56]:
#Pkg.add("Plots")
using Plots
plotly()

The following code loops over all $R_{a,b}$ and computes the terms above. The arrays will be used in subsequent code blocks.

In [78]:
gammaprimeds = Float64[]
R_a_bs = Float64[]
for index = unknown
    a, b = index
    R_a_b = sim_index_reads[index]
    push!(R_a_bs, R_a_b)
    term1_sum =  0.0
    for (aa, j) = filter(i1_i2 -> i1_i2[1] != a, keys(sim_index_reads))
        for (m, bb) = filter(i1_i2 -> i1_i2[2] == b, keys(sim_index_reads))
            term1_sum += sim_index_reads[(a,j)] * sim_index_reads[(m,b)]
        end
    end
    term2_sum =  0.0
    for (i, bb) = filter(i1_i2 -> i1_i2[2] != b, keys(sim_index_reads))
        for (aa, m) = filter(i1_i2 -> i1_i2[1] == a, keys(sim_index_reads))
            term2_sum += sim_index_reads[(i,b)] * sim_index_reads[(a,m)]
        end
    end
    push!(gammaprimeds, R_a_b / (term1_sum + term2_sum))
end
histogram(gammaprimeds, bins=5)

The plot above is perhaps is not easy to interpret since it shows the average $\gamma'$.

### Normalisation 
